In [154]:
import os

def convertToTxt(path):
    for i in os.listdir(path):
        os.rename(dir + "/" + i, dir + "/" + i + ".txt")
        


In [155]:
#get all files in directory
def filesInDirectory(path):
    files = []
    for i in os.listdir(path):
        files.append(path  + "/" + i)
    return files


In [156]:
#Now using pandas we will read each data trail and save it to an array of datasets that contain the eeg singal for each channel
import pandas as pd
import numpy as np


In [157]:

def collectTrialData(TrialNumber):
    with open(SubjectTrialDataset[TrialNumber]) as f:
        lines = f.readlines()

    del lines[0:4]
    count = 0
    valsPerCol = []
    channels = [[0]* 256 for _ in range(64)]
    
    i = 0  
    for line in lines:
        if(line[0] == "#"):   
            line = line.replace("#", "")
            line = line.replace("\n", "")
            line = line.replace(",", "")
            line = line.lstrip()
            channels[i][0] = line
            i += 1  
        #print(count)
    #print(channels)
    currChan = -1
    for line in lines:
        count += 1
        count = count % 257
        if(count == 1):
            #print(valsPerCol)
            currChan += 1
            z = 0
        #Data from cols
        
        if(count != 1):
            z += 1
            work = line.split(" ")
            #print(z-1)
            b = 0
            for w in work:
                b += 1
                b = b % 4
                if(b == 0):
                
                    channels[currChan][z-1] = w

    #Channel[channelNumber][Sample Number] = eegSignal
    #print(channels[0][0])
    return channels


In [158]:
#Master Dir
MasterDirectory = []


In [159]:
#Generate an array of all subjects
peopleDir = []
def getPeople(path):
    for b in os.listdir(path):
        peopleDir.append(b)
       # print(b)
    return peopleDir


In [160]:
#This method is to store the data from every single subject into a dataset we can view
path = "Subjects"

#listOfPeople = []
listOfPeople = getPeople(path)

#Contains an array of each subject trial number[Varies]
listOfTrials = []

#Conditions 
conditionVals = []

for people in listOfPeople:
    listOfTrials = filesInDirectory(path)

info = []
#Get trialData from each person
counter = 0
for a in listOfTrials:
    counter += 1
    allData = []
    #print(a)
    SubjectTrialDataset = filesInDirectory(a)
    #print(SubjectTrialDataset)
    for i in range(len(SubjectTrialDataset)):
        info.append(collectTrialData(i))
    MasterDirectory.append(info)
    

MasterDirectory = info

#MasterDirectory[1]
#print(listOfTrials)
        #this method gets the number of individual trials



#This contains an array of each trial for a subject




#Data set is stored in a matrix where first value is column and second value is sample number
# FP1 FP2 ...
# -1  -3.2
# ..  .. 

#print(SubjectTrialDataset[0])
#This method generates all trial data for set person
#listOfAllTrials = []
#change 30 this is not always the case
#for i in range(30):
#    listOfAllTrials.append(collectTrialData(i))




#val = listOfAllTrials[1]
#print(val[0][0])


IndexError: list index out of range

In [ ]:
#MASTER Array of all subjects [Subject number 1-122]
    ##Contains an array of each subject trial number[Varies]
        ##Contains Conditions (3 columns [1 is single object][2 is 2 object matching][3 is 2 object no match])
            ##Contains matrix that has [Channel][Sample number]

#Solution array of alcoholic/Control
